In [ ]:
import os
import sys
from pathlib import Path

sys.path.append(os.getcwd())
sys.path.append(Path(os.getcwd()).parent.as_posix())

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from perspective_transform.position import positions

In [ ]:
cam_id = "c010"
positions[cam_id]

In [ ]:
img_paths = {
    "c009": "/home/jongphago/nota/frames/val/scene_042/camera09/NIA_MTMDC_s42_c09_am_sunny_fall_0000.jpg",
    "c010": "/home/jongphago/nota/frames/val/scene_042/camera10/NIA_MTMDC_s42_c10_am_sunny_fall_0000.jpg",
}

In [ ]:
_map_image = cv2.imread("/home/jongphago/nota/maps/val/scene_042/map.png")
_cam_image = cv2.imread(img_paths[cam_id])

In [ ]:
# draw cam_position on the cam_image
cam_position = positions[cam_id]["cam_position"]
cam_image = _cam_image.copy()
for index, cp in enumerate(cam_position):
    cv2.circle(cam_image, cp, 10, (0, 0, 255), -1)
    cv2.putText(cam_image, str(index), cp, cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
cv2.polylines(
    cam_image, [np.array(cam_position)], isClosed=True, color=(0, 0, 255), thickness=2
)
plt.imshow(cv2.cvtColor(cam_image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# draw map_position on the map_image
map_position = positions[cam_id]["map_position"]
map_image = _map_image.copy()
for index, cp in enumerate(map_position):
    cv2.circle(map_image, cp, 10, (0, 0, 255), -1)
    cv2.putText(map_image, str(index), cp, cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)
cv2.polylines(
    map_image, [np.array(map_position)], isClosed=True, color=(0, 0, 255), thickness=2
)
plt.imshow(cv2.cvtColor(map_image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
H, _ = cv2.findHomography(np.array(cam_position), np.array(map_position))
H.tolist() == positions[cam_id]["H_inv"]

In [ ]:
H_inv, _ = cv2.findHomography(np.array(map_position), np.array(cam_position))
H_inv.tolist() == positions[cam_id]["H"]

In [ ]:
# cam image에 perspective transform 적용
warped_image = _cam_image.copy()
warped_image = cv2.warpPerspective(
    cam_image, H, (cam_image.shape[1], cam_image.shape[0])
)
plt.imshow(cv2.cvtColor(warped_image, cv2.COLOR_BGR2RGB))

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


class DrawBouondingBox:
    def __init__(self, image, text, tlbr, track_id, colors, m=2):
        self.m = m
        self.tlbr = tlbr.astype(int)
        self.image = image
        self.colors = colors
        self.track_id = track_id
        self.initialize_text(text)

    def initialize_text(self, text, font=None):
        self.text = text
        self.font = font if font is not None else cv2.FONT_HERSHEY_SIMPLEX
        self.text_size = cv2.getTextSize(
            self.text, self.font, 0.4 * self.m, 1 * self.m
        )[0]
        self.text_background_color = (
            (self.colors[self.track_id % 80] * 255 * 0.7).astype(np.uint8).tolist()
        )
        self.text_color = (
            (0, 0, 0)
            if np.mean(self.colors[self.track_id % 80]) > 0.5
            else (255, 255, 255)
        )

    def draw_text_background(self):
        x0, y0 = self.tlbr[:2]
        pt1 = (x0, y0 - 2)
        pt2 = (x0 + self.text_size[0] + 1, y0 - int(1.5 * self.text_size[1]))
        cv2.rectangle(self.image, pt1, pt2, self.text_background_color, -1)

    def draw_text(self):
        x0, y0 = self.tlbr[:2]
        org = (x0, int(y0 - self.text_size[1] / 2))
        font_scale = 0.4 * self.m
        thickness = 1 * self.m
        cv2.putText(
            self.image,
            self.text,
            org,
            self.font,
            font_scale,
            self.text_color,
            thickness=thickness,
        )

    def draw_label(self):
        self.draw_text_background()
        self.draw_text()

    def draw_bounding_box(self, line_thickness=2):
        color = (self.colors[self.track_id % 80] * 255).astype(np.uint8).tolist()
        top_left, bottom_right = self.tlbr[:2], self.tlbr[2:]
        cv2.rectangle(self.image, top_left, bottom_right, color, line_thickness)

    def draw(self):
        self.draw_bounding_box()
        self.draw_label()


_img = np.random.randint(0, 255, (1080, 1920, 3)).astype(np.uint8)
img_paths = [1]
path_tlbr = {1: np.array([408.79, 547.86, 501.56, 840.7])}
track_id = 0
m = 2
text = f"{track_id}"
txt_color = (255, 255, 255)
colors = [np.array((0, 0, 255))]
color = np.array((0, 0, 255))
font = cv2.FONT_HERSHEY_SIMPLEX
txt_size = cv2.getTextSize(text, font, 0.4 * m, 1 * m)[0]
txt_bk_color = (color * 0.7).astype(np.uint8).tolist()
color = (color).astype(np.uint8).tolist()


for img_path in img_paths:
    draw_bounding_box = DrawBouondingBox(
        _img.copy(), text, path_tlbr[img_path], track_id, colors
    )
    draw_bounding_box.draw()


plt.imshow(draw_bounding_box.image)

In [ ]:
class DrawMap:
    def __init__(self, map_image, track, tracker_index, colors):
        self.map_image = map_image
        self.track = track
        self.initialize_track(colors)
        self.initialize_text(tracker_index)

    def initialize_track(self, colors):
        self.location = tuple(map(int, self.track.location[0]))
        self.track_id = self.track.global_id
        self.color = (colors[self.track_id % 80] * 255).astype(np.uint8).tolist()

    def initialize_text(self, tracker_index):
        self.text = f"{str(track_id)}[{tracker_index}]"
        self.font_face = cv2.FONT_HERSHEY_SIMPLEX
        self.font_scale = 1

    def draw_map_position(self):
        radius = 10
        thickness = -1
        cv2.circle(
            self.map_image,
            self.location,
            radius,
            self.color,
            thickness,
        )

    def draw_map_label(self, thickness=2):
        cv2.putText(
            self.map_image,
            self.text,
            self.location,
            self.font_face,
            self.font_scale,
            self.color,
            thickness,
        )

    def draw(self):
        self.draw_map_position()
        self.draw_map_label()

    def get_map_image(self):
        return self.map_image